In [14]:
import zipfile
import os

#dearhivare

uploaded_file_path = "ML_dataset.zip"
extraction_dir = "ML_dataset"

with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

print(f"Files extracted to: {extraction_dir}")

Files extracted to: ML_dataset


In [13]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

base_dir = "ML_dataset/ML_dataset"
directories = ['S', 'Z']
labels_map = {'S': 1, 'Z': 0}  #1 = epileptic 0 = healthy

X, y = [], [] #lists
max_length = 1000  #fixed length for all time series

for folder_name in directories:
    folder_path = os.path.join(base_dir, folder_name)   #to know in what subdirectory we are
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        print(file_path)
        data = np.loadtxt(file_path)  #time series represented by the files
        X.append(data) #data from each file goes into input
        y.append(labels_map[folder_name]) #adds labels

X = pad_sequences(X, maxlen=max_length, padding='post', truncating='post') #all timeseries become 1000 numbers long
y = np.array(y) #for ML np.array more efficient than lists

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=60)

#simole feedforward nn
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_length,)),  #input layer
    tf.keras.layers.Dense(16, activation='relu'), #hidden layer (16 neurons)
    tf.keras.layers.Dense(1, activation='sigmoid') #output layer (1 neuron)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

predictions = model.predict(X_test)
predicted_labels = (predictions > 0.5).astype(int) #converts probabilities given by sigmoid function into 0 and 1; false=0 (healthy) and true=1 (epileptic)


ML_dataset/ML_dataset/S/S036.txt
ML_dataset/ML_dataset/S/S028.txt
ML_dataset/ML_dataset/S/S030.txt
ML_dataset/ML_dataset/S/S099.txt
ML_dataset/ML_dataset/S/S057.txt
ML_dataset/ML_dataset/S/S082.txt
ML_dataset/ML_dataset/S/S054.txt
ML_dataset/ML_dataset/S/S090.txt
ML_dataset/ML_dataset/S/S010.txt
ML_dataset/ML_dataset/S/S011.txt
ML_dataset/ML_dataset/S/S038.txt
ML_dataset/ML_dataset/S/S001.txt
ML_dataset/ML_dataset/S/S056.txt
ML_dataset/ML_dataset/S/S071.txt
ML_dataset/ML_dataset/S/S008.txt
ML_dataset/ML_dataset/S/S009.txt
ML_dataset/ML_dataset/S/S052.txt
ML_dataset/ML_dataset/S/S067.txt
ML_dataset/ML_dataset/S/S031.txt
ML_dataset/ML_dataset/S/S051.txt
ML_dataset/ML_dataset/S/S098.txt
ML_dataset/ML_dataset/S/S065.txt
ML_dataset/ML_dataset/S/S007.txt
ML_dataset/ML_dataset/S/S092.txt
ML_dataset/ML_dataset/S/S025.txt
ML_dataset/ML_dataset/S/S026.txt
ML_dataset/ML_dataset/S/S061.txt
ML_dataset/ML_dataset/S/S049.txt
ML_dataset/ML_dataset/S/S094.txt
ML_dataset/ML_dataset/S/S078.txt
ML_dataset